In [1]:

import networkx as nx
import numpy as np
import grakel as gk
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
import tqdm


import sys
sys.path.append("../")
import MMDforGraphs as mg
from importlib import reload  
foo = reload(mg)
import myKernels.RandomWalk as rw

In [34]:
np.random.seed(seed=42)
k = 5
sparsity = .9
# generate the symmetric sparsity mask
mask = np.random.uniform(size = k)
mask = mask * (mask < sparsity)
mask = np.triu(mask)
mask = mask + mask.T + np.identity(k)
mask[mask > 0] = 1

# generate the symmetric precision matrix
theta = np.random.normal(size = (k,k))
theta = np.random.normal(size = (k,k))
theta = np.triu(theta)
theta = theta + theta.T + np.identity(k)

# apply the reqired sparsity
theta = theta * mask

l, _ = np.linalg.eigh(theta)
# force it to be positive definite
theta = theta - (np.min(l)-.1) * np.identity(k)

In [33]:
theta

array([[ 3.31578085,  0.        ,  0.        , -0.22945045,  0.38934891],
       [ 0.        ,  5.82327615,  0.        ,  1.19363972,  0.21863832],
       [ 0.        ,  0.        ,  0.4727032 ,  0.77370042, -0.53814166],
       [-0.22945045,  1.19363972,  0.77370042,  3.90814939,  0.58212279],
       [ 0.38934891,  0.21863832, -0.53814166,  0.58212279,  2.39233684]])

In [35]:
theta

array([[ 4.25874059e+00,  0.00000000e+00, -5.52786232e-03,
        -2.29450454e-01,  3.89348913e-01],
       [ 0.00000000e+00,  6.76623590e+00,  2.77831304e+00,
         1.19363972e+00,  2.18638316e-01],
       [-5.52786232e-03,  2.77831304e+00,  1.41566294e+00,
         7.73700417e-01, -5.38141662e-01],
       [-2.29450454e-01,  1.19363972e+00,  7.73700417e-01,
         4.85110913e+00,  5.82122795e-01],
       [ 3.89348913e-01,  2.18638316e-01, -5.38141662e-01,
         5.82122795e-01,  3.33529658e+00]])

In [2]:
bg1 = mg.TemporalBinomialGraphs(n = 100, nnode = 40, k = 4, ar = 0.9, l = 'degreelabels', fullyConnected = True)
bg2 = mg.TemporalBinomialGraphs(n = 100, nnode = 40, k = 4, ar = 0.7, l = 'degreelabels', fullyConnected = True)

In [4]:
# importlib.reload(sys.modules['MMDforGraphs'])
bg1.Generate()
bg2.Generate()
Gs = bg1.Gs + bg2.Gs

# Grakel kernels
graph_list = gk.graph_from_networkx(Gs, node_labels_tag='label')  # Convert to graphs to Grakel format
# kernel = [{"name": "multiscale_laplacian"}]
# K = mg.KernelMatrix(graph_list, kernel, False)   
#kernel = [{"name": "weisfeiler_lehman", "n_iter": 4}, {"name": "vertex_histogram"}]
kernel = [{"name": "WL-OA", "n_iter": 4}]
# kernel = [{"name": "SP", "with_labels": 1}]
# kernel = [{"name": "lovasz_theta", "n_samples":20}]
#kernel = [{"name": "core_framework", "min_core": 1}, {"name":"propagation", 't_max':5, 'w':0.1, 'M':"H"}]
# kernel = [{"name": "pyramid_match", "with_labels":False, "d":2, "L":4}] 
#kernel = [{"name":'RW', 'kernel_type':'geometric', 'lamda':0.1, 'with_labels':False, 'p':3}]
# kernel = [{"name":'odd_sth', 'h':3}]
#kernel = [{"name": "weisfeiler_lehman", "n_iter": 1}, {"name": "propagation", "t_max": 2, "w":0.01, "M":'TV'}]

init_kernel = gk.GraphKernel(kernel= kernel, normalize=1, n_jobs= 4)
#K = init_kernel.fit_transform(graph_list)

# My kernels
# RW kernel
# label list for rw
label_list = []
for G in Gs:
    label_list.append(np.unique(list(nx.get_node_attributes(G, 'label').values())))

# label_list = np.unique(np.concatenate(label_list))
rw_kernel = rw.RandomWalk(Gs, c = 0.01, normalize=0 )
# K = rw_kernel.fit_ARKL(r = 8, label_list = label_list, normalize_adj=True)
K = rw_kernel.fit_ARKU_plus(r = 12, normalize_adj=False)
# K = rw_kernel.fit_random_walk(mu_vec=[0.0, 1.0/(nnode_1*nnode_2)], k = 1, r = 3, normalize_adj=False ) # 1/((nnode_1-1)*(nnode_2-1)))

print(np.all(K == 0)) 
print(K)


MMD_functions = [mg.MMD_b, mg.MMD_u]

# initialize bootstrap class, we only want this to be initalized once so that numba njit
# only gets compiled once (at first call)
kernel_hypothesis = mg.BoostrapMethods(MMD_functions)
function_arguments=[dict(n = bg1.n, m = bg2.n ), dict(n = bg1.n, m = bg2.n )]
kernel_hypothesis.Bootstrap(K, function_arguments, B = 3000)

# Run power
#threshold = np.quantile(kernel_hypothesis.boot_test_statistic['MMD_u'], 1-0.05)
#print(f'threshold {threshold}')
#ratio, power, V = mg.power_ratio(K, kernel_hypothesis.sample_test_statistic['MMD_u'], threshold, n_1)
#print(f'V {V}')
#print(f'ratio {ratio}')
#print(f'power {power}')
print(f'p_value {kernel_hypothesis.p_values}')
print(f"MMD_u {kernel_hypothesis.sample_test_statistic['MMD_u']}")

  0%|          | 0/20100.0 [00:00<?, ?it/s]..\myKernels\RandomWalk.py:903: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return scipy.sparse.csr_matrix(nx.linalg.adjacency_matrix(G), dtype=np.float64)
100%|██████████| 20100/20100.0 [02:58<00:00, 112.43it/s]


False
[[0.00077071 0.00079574 0.00079923 ... 0.00074435 0.00075826 0.00076458]
 [0.00079574 0.00082546 0.00082997 ... 0.00076428 0.00078077 0.00078858]
 [0.00079923 0.00082997 0.00083503 ... 0.00076654 0.00078357 0.00079196]
 ...
 [0.00074435 0.00076428 0.00076654 ... 0.0007236  0.00073471 0.00073928]
 [0.00075826 0.00078077 0.00078357 ... 0.00073471 0.00074724 0.00075262]
 [0.00076458 0.00078858 0.00079196 ... 0.00073928 0.00075262 0.0007587 ]]
p_value {'MMD_b': 0.207, 'MMD_u': 0.207}
MMD_u 5.192274661744087e-08
